<a href="https://colab.research.google.com/github/justmonis/Covid19_india_Exploratory-Data-Analysis-with-covid19_india-Data/blob/main/Exploratory_Data_Analysis_with_covid19_india_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'covid-19-india2021:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2459355%2F4167029%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240410%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240410T133305Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D271fd45e55e6a98f6d38ad77dc8781d9350918f6be3da545e9959d9287156b97185ee4994fd80e2f7962fbb515ed92753852220eb4636710c659c3bfae5d727345b8b5b73ca5dac06021b662c47275048ab575d109e41e4b9b2a570780dd27a0417025be2a17d8b096faa72441237db63dcd090b8f0e3b4cd985c784886522cb638ec0f6b3ce537222ef8b861db9fae442b5418c6229e27f130ee606ea4f240913222cf99a294b3b984d3ad4fbe4cb111ab96fe1cac8bde3314b0a1927eae719b3bf1c1c222cc858eac21e4546c71acab444d44317016c6dbfa47dc9792b584cbce1fe436eedfbfb97c28a1de00a4ee0c71f9b4167c23dde8eeb3401427c7ef5'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import datetime as dt
import numpy as np


In [ ]:
#importing main dataset
df = pd.read_csv('../input/covid-19-india2021/covid_19_india.csv',parse_dates=['Date'],dayfirst=True)

In [ ]:
df.head()

In [ ]:
#keeping only required columns
df = df[['Date','State/UnionTerritory','Cured','Deaths','Confirmed']]
#renaming column names
df.columns = ['date','state','cured','deaths','confirmed']

In [ ]:
#looking at the earlier dates
df.head()

In [ ]:
#Looking at the Latest dates
df.tail()

In [ ]:
#current date
today = df[df.date == '2020-07-17']

In [ ]:
today

In [ ]:
today.head()

In [ ]:
#Sorting data w.r.t number of confirmed cases
max_confirmed_cases=today.sort_values(by="confirmed",ascending=False)
max_confirmed_cases

In [ ]:
#Getting states with maximum number of confirmed cases
top_states_confirmed=max_confirmed_cases[0:5]

In [ ]:
top_states_confirmed

In [ ]:
#Making bar-plot for states with top confirmed cases
sns.set(rc={'figure.figsize':(15,10)})
sns.barplot(x="state",y="confirmed",data=top_states_confirmed,hue="state")
plt.show()

In [ ]:
#Sorting data w.r.t number of death cases
max_death_cases=today.sort_values(by="deaths",ascending=False)
max_death_cases

In [ ]:
#Getting states with maximum number of death cases
top_states_death=max_death_cases[0:5]

In [ ]:
#Making bar-plot for states with top death cases
sns.set(rc={'figure.figsize':(15,10)})
sns.barplot(x="state",y="deaths",data=top_states_death,hue="state")
plt.show()

In [ ]:
#Sorting data w.r.t number of cured cases
max_cured_cases=today.sort_values(by="cured",ascending=False)
max_cured_cases

In [ ]:
#Getting states with maximum number of cured cases
top_states_cured=max_cured_cases[0:5]

In [ ]:
#Making bar-plot for states with top death cases
sns.set(rc={'figure.figsize':(15,10)})
sns.barplot(x="state",y="cured",data=top_states_cured,hue="state")
plt.show()

In [ ]:
#Maharashtra
maha = df[df.state == 'Maharashtra']

In [ ]:
maha

In [ ]:
#Visualizing confirmed cases in maharashtra
sns.set(rc={'figure.figsize':(15,10)})
sns.lineplot(x="date",y="confirmed",data=maha,color="c")
plt.show()

In [ ]:
#Visualizing death cases in maharashtra
sns.set(rc={'figure.figsize':(15,10)})
sns.lineplot(x="date",y="deaths",data=maha,color="y")
plt.show()

In [ ]:
#kerala
kerala = df[df.state == 'Kerala']
kerala

In [ ]:
#Visualizing confirmed cases in Kerala
sns.set(rc={'figure.figsize':(15,10)})
sns.lineplot(x="date",y="confirmed",data=kerala,color="#00ffcc")
plt.show()

In [ ]:
#Visualizing confirmed cases in Kerala
sns.set(rc={'figure.figsize':(15,10)})
sns.lineplot(x="date",y="deaths",data=kerala,color="#000000")
plt.show()

In [ ]:
#Jammu and Kashmir
jk = df[df.state == 'Jammu and Kashmir']
jk

In [ ]:
#Visualizing confirmed cases in Jammu and Kashmir
sns.set(rc={'figure.figsize':(15,10)})
sns.lineplot(x="date",y="confirmed",data=jk,color="#cc66ff")
plt.show()

In [ ]:
#Visualizing confirmed cases in jk
sns.set(rc={'figure.figsize':(15,10)})
sns.lineplot(x="date",y="deaths",data=jk,color="#000099")
plt.show()

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly as py
# py.offline.init_notebook_mode(connected = True)
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import folium
import math
import random
from datetime import timedelta
import warnings
warnings.filterwarnings('ignore')
# Color pallatte
cnf = '#393e46'
dth = '#ff2e63'
rec = '#21bf73'
act = '#fe9801'

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))